In [1]:
import os
import re
import glob
import json
import csv

def check_text_quality(text, min_length=500, min_alpha_ratio=0.7, verbose=False):
    quality = {
        "length": len(text),
        "alnum_ratio": len(re.findall(r'\w', text)) / max(len(text), 1),
        "long_word_count": len([w for w in text.split() if len(w) > 3]),
    }

    quality["ok"] = (
        quality["length"] >= min_length and
        quality["alnum_ratio"] >= min_alpha_ratio and
        quality["long_word_count"] >= 10
    )

    if verbose and not quality["ok"]:
        print(f"⚠️ Quality check failed: {quality}")

    return quality

def clean_text(text):
    text = re.sub(r'(.)\1{3,}', r'\1', text)  # Collapse repeated characters
    text = text.replace('ﬁ', 'fi').replace('ﬂ', 'fl')  # Fix ligatures
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # Remove non-ASCII
    text = re.sub(r'-\s*\n\s*', '', text)  # Remove hyphenation at line breaks
    text = re.sub(r'\s+', ' ', text).strip()  # Normalize whitespace
    return text

def process_texts(text_dir="texts", cleaned_dir="texts_cleaned",
                  low_quality_log="low_quality_files.json",
                  stats_csv="text_quality_stats.csv",
                  skip_low_quality=True):
    os.makedirs(cleaned_dir, exist_ok=True)

    # Load low quality log if it exists
    low_quality = []
    if os.path.exists(low_quality_log):
        with open(low_quality_log, "r") as f:
            low_quality = json.load(f)

    cleaned_files = {os.path.basename(f) for f in glob.glob(os.path.join(cleaned_dir, "*.txt"))}
    files = glob.glob(os.path.join(text_dir, "*.txt"))
    print(f"🔍 Found {len(files)} files in '{text_dir}'")

    stats = []

    for path in files:
        filename = os.path.basename(path)
        if filename in cleaned_files:
            print(f"[SKIP] Already cleaned: {filename}")
            continue

        with open(path, "r", encoding="utf-8") as f:
            raw_text = f.read()

        cleaned = clean_text(raw_text)
        quality = check_text_quality(cleaned, verbose=True)
        quality["filename"] = filename
        stats.append(quality)

        if skip_low_quality and not quality["ok"]:
            print(f"[LOW QUALITY] {filename} — skipping")
            low_quality.append(filename)
            continue

        out_path = os.path.join(cleaned_dir, filename)
        with open(out_path, "w", encoding="utf-8") as f:
            f.write(cleaned)
        print(f"[CLEANED] {filename}")

    # Deduplicate and save low quality log
    low_quality = sorted(set(low_quality))
    with open(low_quality_log, "w") as f:
        json.dump(low_quality, f, indent=2)

    print(f"\n📉 Logged {len(low_quality)} low-quality files to {low_quality_log}")

    # Write CSV of stats
    if stats:
        keys = ["filename", "length", "alnum_ratio", "long_word_count", "ok"]
        with open(stats_csv, "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=keys)
            writer.writeheader()
            writer.writerows(stats)
        print(f"📊 Saved quality stats to {stats_csv}")

if __name__ == "__main__":
    process_texts()


🔍 Found 11976 files in 'texts'
[SKIP] Already cleaned: Malyun-Karama-2020-0162_Redacted.pdf.txt
[SKIP] Already cleaned: Theresa-Robertson-2020-0158_Redacted.pdf.txt
[SKIP] Already cleaned: George-Townsend-2020-0157_Redacted.pdf.txt
[SKIP] Already cleaned: Jerrelle-McKenzie-2020-0144_Redacted.pdf.txt
[SKIP] Already cleaned: Joan-Williams-2020-0128_Redacted.pdf.txt
[SKIP] Already cleaned: Lynda-Pedersen-2020-0112.pdf.txt
[SKIP] Already cleaned: Tomasz-Nowasad-2019-0445_Redacted.pdf.txt
[SKIP] Already cleaned: Keith-WHETTON-2019-0452_Redacted.pdf.txt
[SKIP] Already cleaned: Kieran-Hubbard-2019-0451_Redacted.pdf.txt
[SKIP] Already cleaned: Barry-Liffen-2019-0440_Redacted.pdf.txt
[SKIP] Already cleaned: Eugeniusz-MALEK-2019-0439_Redacted.pdf.txt
[SKIP] Already cleaned: Peter-Frosdick-2019-0423_Redacted.pdf.txt
[SKIP] Already cleaned: 2019-0378-Royal-College-of-General-Practitioners.pdf.txt
[SKIP] Already cleaned: David-Kirsch-2019-0362_Redacted.pdf.txt
[SKIP] Already cleaned: Alex-Malcolm-2

[SKIP] Already cleaned: 2018-0173-Kernow-NHS-Trust.pdf.txt
[SKIP] Already cleaned: Elaine-Horrocks-2018-0169_Redacted.pdf.txt
[SKIP] Already cleaned: Mwitumwa-Ngenda-2018-0167_Redacted.pdf.txt
[SKIP] Already cleaned: Michalla-Sweeting-2018-0165_Redacted.pdf.txt
[SKIP] Already cleaned: Alfie-Scambler-Holt-2018-0156_Redacted.pdf.txt
[SKIP] Already cleaned: George-Goldby-2018-0104_Redacted.pdf.txt
[SKIP] Already cleaned: Harry-Jellicoe-2018-0108_Redacted.pdf.txt
[SKIP] Already cleaned: Karen-Edgar-2018-0106_Redacted.pdf.txt
[SKIP] Already cleaned: Lakhminder-Kaur-2018-0029.pdf.txt
[SKIP] Already cleaned: Bethany-Shipsey-2018-0049_Redacted.pdf.txt
[SKIP] Already cleaned: Riaz-Begum-2018-0041_Redacted.pdf.txt
[SKIP] Already cleaned: David-Buttriss-2018-0010_Redacted.pdf.txt
[SKIP] Already cleaned: Rose-Ball-2017-0395_Redacted.pdf.txt
[SKIP] Already cleaned: Anthony-Grant-2017-0410_Redacted.pdf.txt
[SKIP] Already cleaned: Robert-Richards-2017-0406_Redacted.pdf.txt
[SKIP] Already cleaned: Mic

[SKIP] Already cleaned: Sousse-Inquest-2017-0206.pdf.txt
[SKIP] Already cleaned: Fairclough-2017-0119.pdf.txt
[SKIP] Already cleaned: Krasinsky-Lloyd-2017-0109.pdf.txt
[SKIP] Already cleaned: FOSTER-2017-0095.pdf.txt
[SKIP] Already cleaned: Hooper-2017-0068_Redacted.pdf.txt
[SKIP] Already cleaned: Bevan-2017-0060.pdf.txt
[SKIP] Already cleaned: Alexander-2017-0044.pdf.txt
[SKIP] Already cleaned: Evanson-2016-0359.pdf.txt
[SKIP] Already cleaned: Lock-2016-0355.pdf.txt
[SKIP] Already cleaned: Bassendine-2016-0424.pdf.txt
[SKIP] Already cleaned: Arthur-2017-0009.pdf.txt
[SKIP] Already cleaned: ODonoghue-2017-0007.pdf.txt
[SKIP] Already cleaned: Mills-2016-0416.pdf.txt
[SKIP] Already cleaned: Charles-2016-0465.pdf.txt
[SKIP] Already cleaned: Haughton-2016-0339.pdf.txt
[SKIP] Already cleaned: Jones-2016-0327.pdf.txt
[SKIP] Already cleaned: Cahill-2016-0304.pdf.txt
[SKIP] Already cleaned: Mindham-2016-0295.pdf.txt
[SKIP] Already cleaned: Olawale-ADELUSI_Redacted.pdf.txt
[SKIP] Already cleane

[SKIP] Already cleaned: 2016-0271-Response-by-University-health-Board.pdf.txt
[SKIP] Already cleaned: 2016-0276-Response-by-Medicines-and-Healthcare-Products-Regulatory-Agency.pdf.txt
[SKIP] Already cleaned: 2016-0303-Response-by-North-West-Ambulance-Service-NHS-Trust.pdf.txt
[SKIP] Already cleaned: 2016-0308-Response-by-Medicines-and-Healthcare-Products-Regulatory-Agency.pdf.txt
[SKIP] Already cleaned: 2016-0316-Response-by-Warwick-Medical-School.pdf.txt
[SKIP] Already cleaned: 2016-0367-Response-by-NMC.pdf.txt
[SKIP] Already cleaned: 2016-0376-Response-by-Resuscitation-Council-UK.pdf.txt
[SKIP] Already cleaned: 2016-0384-Response-by-Yorkshire-Ambulance-Service.pdf.txt
[SKIP] Already cleaned: 2016-0396-Response-by-Maritime-and-Coastguard-Agency.pdf.txt
[SKIP] Already cleaned: 2016-0402-Response-by-RYA.pdf.txt
[SKIP] Already cleaned: 2016-0459-Response-by-University-Health-Board.pdf.txt
[SKIP] Already cleaned: 2016-0600-Response-from-the-London-Ambulance-Service_Redacted.pdf.txt
[SKIP]

[SKIP] Already cleaned: 2017-0193-Response-by-NMC.pdf.txt
[SKIP] Already cleaned: 2017-0234-Response-by-The-Royal-College-of-Emergency-Medicine.pdf.txt
[SKIP] Already cleaned: 2017-0248-Response-by-Priory-Hospital_Redacted.pdf.txt
[SKIP] Already cleaned: 2017-0275-Response-by-Parkhill-Estates-Limited.pdf.txt
[SKIP] Already cleaned: 2017-0325-Response-by-Bedfordshire-Police.pdf.txt
[SKIP] Already cleaned: 2017-0348-Response-United-Lincolnshire-Hospitals-NHS-Trust.pdf.txt
[SKIP] Already cleaned: 2017-0376-Response-by-East-Midlands-Ambulance-Service-NHS-Trust.pdf.txt
[SKIP] Already cleaned: 2017-0434-Response-by-Draeger-Medical-UK-Limited.pdf.txt
[SKIP] Already cleaned: 2018-0007-Response.pdf.txt
[SKIP] Already cleaned: 2018-0008-Response-by-CAA.pdf.txt
[SKIP] Already cleaned: 2018-0021-Response-by-Sussex-Partnership-NHS-Trust.pdf.txt
[SKIP] Already cleaned: 2018-0066-Response-by-University-Health-Board.pdf.txt
[SKIP] Already cleaned: 2018-0084-Responses.pdf.txt
[SKIP] Already cleaned: 20

...

[CLEANED] Timothy-De-Boos-Prevention-of-Future-Deaths-Report-2024-0691.pdf.txt
[SKIP] Already cleaned: 2024-0691-Response-from-DHSC.pdf.txt
[SKIP] Already cleaned: Matthew-Sheldrick-2-Prevention-of-Future-Deaths-Report-2024-0690.pdf.txt
[SKIP] Already cleaned: 2024-0690-Response-from-NHS-England.pdf.txt
[SKIP] Already cleaned: 2024-0690-Response-from-DHSC.pdf.txt
[SKIP] Already cleaned: Matthew-Sheldrick-1-Prevention-of-Future-Deaths-Report-2024-0689.pdf.txt
[SKIP] Already cleaned: 2024-0689-Response-from-Sussex-ICB.pdf.txt
[SKIP] Already cleaned: Laura-Jane-Seaman-Prevention-of-Future-Deaths-Report-2024-0688.pdf.txt
[SKIP] Already cleaned: 2024-0688-Response-from-Royal-College-of-Obstetricians-and-Gynaecologists.pdf.txt
[SKIP] Already cleaned: 2024-0688-Response-from-Mid-South-Essex-NHS-Trust.pdf.txt
[SKIP] Already cleaned: Susan-Evans-Prevention-of-Future-Deaths-Report-2024-0687.pdf.txt
[SKIP] Already cleaned: 2024-0687-Response-from-Portsmouth-Hospital-NHS-Trust.pdf.txt
[SKIP] Alrea

[CLEANED] 2024-0643-Response-from-Mid-and-West-Wales-Fire-and-Rescue-Service.pdf.txt
[SKIP] Already cleaned: 2024-0643-Response-from-Welsh-Government.pdf.txt
[SKIP] Already cleaned: Dorothy-Nias-Prevention-of-Future-Deaths-Report-2024-0642.pdf.txt
[SKIP] Already cleaned: 2024-0642-Response-from-DVLA.pdf.txt
[SKIP] Already cleaned: 2024-0642-Response-from-Department-for-Transport.pdf.txt
[SKIP] Already cleaned: Kevin-Ince-Prevention-of-Future-Deaths-Report-2024-0641.pdf.txt
[SKIP] Already cleaned: 2024-0641-Response-from-The-Priory.pdf.txt
[SKIP] Already cleaned: Edward-Barnard-Prevention-of-Future-Deaths-Report-2024-0640.pdf.txt
[SKIP] Already cleaned: 2024-0640-Response-from-RCVS.pdf.txt
[SKIP] Already cleaned: 2024-0640-Veterinary-Medicines-Directorate.pdf.txt
[SKIP] Already cleaned: Charlotte-Roscoe-Prevention-of-Future-Deaths-Report-2024-0639.pdf.txt
[SKIP] Already cleaned: 2024-0639-Response-from-The-Royal-College-of-Radiologists.pdf.txt
[SKIP] Already cleaned: 2024-0639-Response-

[CLEANED] Mark-Eccles-Prevention-of-Future-Deaths-Report-2024-0579.pdf.txt
[SKIP] Already cleaned: 2024-0579-Response-from-Herefordshire-Council.pdf.txt
[SKIP] Already cleaned: Chloe-Every-Prevention-of-Future-Deaths-Report-2024-0578.pdf.txt
[SKIP] Already cleaned: 2024-0578-Response-from-Barking-Havering-and-Redbridge-NHS-Foundation-Trust.pdf.txt
[SKIP] Already cleaned: 2024-0578-Response-from-DHSC.pdf.txt
[SKIP] Already cleaned: Mark-Beresford-Prevention-of-Future-Deaths-Report-2024-0577.pdf.txt
[SKIP] Already cleaned: 2024-0577-Response-from-HMPPS.pdf.txt
[SKIP] Already cleaned: Sylvia-Prichard-Prevention-of-Future-Deaths-Report-2024-0576.pdf.txt
[CLEANED] 2024-0576-Response-from-Avery-Healthcare-Group.pdf.txt
[SKIP] Already cleaned: 2024-0575-Response-from-NHS-Stockport.pdf.txt
[SKIP] Already cleaned: Charles-Daniels-Prevention-of-Future-Deaths-Report-2024-0575.pdf.txt
[SKIP] Already cleaned: Patricia-Lines-Prevention-of-Future-Deaths-Report-2024-0574.pdf.txt
[SKIP] Already cleaned

[CLEANED] 2024-0429-Response-from-Westmorland-Court-Care-Home.pdf.txt
[CLEANED] 2024-0429-Response-from-NMC.pdf.txt
[CLEANED] 2024-0429-Response-from-CQC.pdf.txt
[SKIP] Already cleaned: 2024-0428-Response-from-Durham-County-Council.pdf.txt
[SKIP] Already cleaned: 2024-0427-Response-from-North-East-Ambulance-Service.pdf.txt
[SKIP] Already cleaned: 2024-0426-Response-from-HSE.pdf.txt
[SKIP] Already cleaned: 2024-0425-Response-from-Department-for-Education.pdf.txt
[SKIP] Already cleaned: 2024-0424-Response-from-ROSPA.pdf.txt
[SKIP] Already cleaned: 2024-0422-Response-from-Birmingham-and-Solihull-Mental-Health-NHS-Foundation-Trust.pdf.txt
[CLEANED] 2024-0421-Response-from-James-Paget-University-Hospitals-page-2.pdf.txt
[CLEANED] 2024-0421-Response-from-James-Paget-University-Hospitals-page-1.pdf.txt
[SKIP] Already cleaned: 2024-0418-Response-from-HMPPS.pdf.txt
[SKIP] Already cleaned: 2024-0416-Response-from-Royal-College-of-Physicians.pdf.txt
[SKIP] Already cleaned: 2024-0416-Response-from

[CLEANED] 2020-0002-Response-from-County-Durham-and-Darlington-NHS.pdf.txt
[CLEANED] 2019-0413-Response-from-Durham-County-Council.pdf.txt
[CLEANED] 2016-0230-Response-from-Cornwall-Council.pdf.txt
[SKIP] Already cleaned: 2014-0061-Response-from-Care-UK.pdf.txt
[SKIP] Already cleaned: Neville-Bardoliwalla-2020-0258.pdf.txt
[SKIP] Already cleaned: John-Jennings-2020-0257.pdf.txt
[CLEANED] Sean-Ennis-Prevention-of-future-deaths-report-2022-0054.pdf.txt
[CLEANED] Barbara-Mitchell-Prevention-of-future-deaths-report-2023-0153_Published.pdf.txt

📉 Logged 2143 low-quality files to low_quality_files.json
📊 Saved quality stats to text_quality_stats.csv


In [2]:
import pandas as pd
df = pd.read_csv('text_quality_stats.csv')

In [11]:
df[~df['ok']]

,filename,length,alnum_ratio,long_word_count,ok
0,2021-0077-Response-from-St-Pancras-Hospital-Re...,0,0.000000,0,False
1,2021-0058-Academy-of-Medical-Royal-Colleges-Re...,250,0.832000,28,False
2,Liane-Davenport-2020-0136_Redacted-1.pdf.txt,0,0.000000,0,False
187,2019-0494-Response-by-Stockport-NHS-Trust.pdf.txt,118,0.754237,11,False
204,2021-0131-Response-from-from-Canal-River-Trust...,125,0.808000,14,False
357,2019-0119-Response-from-the-Department-for-Tra...,1473,0.397828,36,False
731,2017-0205-Response-by-South-London-and-Maudsle...,3031,0.690861,137,False
1082,2022-0233-Response-from-Royal-College-General-...,0,0.000000,0,False
1085,2022-0123-Response-from-British-Standards-Inst...,8358,0.562216,58,False
1086,2022-0101-Response-from-Spire-Healthcare.pdf.txt,0,0.000000,0,False


In [17]:
df[df['filename'].str.contains("Andrea-Mann-Prevention-of-Future-Deaths-Report")].filename.values

array(['Andrea-Mann-Prevention-of-Future-Deaths-Report-2025-0130-1.pdf.txt'],
      dtype=object)

In [18]:
with open("texts/Andrea-Mann-Prevention-of-Future-Deaths-Report-2025-0130-1.pdf.txt", "r", encoding="utf-8") as f:
    print(f.read())

, Chief Executive (Bradford District Care NHS Trust)




In [19]:
import pdfplumber

with pdfplumber.open("downloads/Andrea-Mann-Prevention-of-Future-Deaths-Report-2025-0130-1.pdf") as pdf:
    full_text = "\n".join([page.extract_text() or "" for page in pdf.pages])

print(full_text[:1500])

, Chief Executive (Bradford District Care NHS Trust)




In [6]:
with open("texts/Shariff-2016-0321.pdf.txt", "r", encoding="utf-8") as f:
    print(f.read())

REGULATION 28: REPORT TO PREVENT FUTURE DEATHS (1)
REGULATION 28 REPORT TO PREVENT FUTURE DEATHS
THIS REPORT IS BEING SENT TO:
1. Department of Health
2. NICE
3. Pennine Acute NHS Trust
4. Chief Coroner
CORONER
I am Ms Julie Robertson, Assistant Coroner for the Coroner area of Manchester North
2 CORONER’S LEGAL POWERS
I make this report under paragraph 7, Schedule 5, of the Coroner’s and Justice Act 2009 and Regulations 28
and 29 of the Coroners (Investigations) Regulations 2013
3 INVESTIGATION and INQUEST
On the 10 May 2016 I commenced an investigation into the death of Dildar Shariff. The inquest into Mr
Shariff’s death was heard on 7 September 2016.
4 CIRCUMSTANCES OF DEATH
Mr Shariff died on 10 May 2016 at Fairfield General Hospital having been admitted following a
cardiac arrest at his home address that day. He had had an unwitnessed fallen from a chair onto
his kitchen floor on 8 May and attended at the Urgent Care Centre promptly following that fall.
Neither a CT scan nor additi

In [7]:
with open("texts/2016-0230-Response-from-Cornwall-Council.pdf.txt", "r", encoding="utf-8") as f:
    print(f.read())

one and all ‘onen hag all

CORNWALL
COUNCIL

Dr E Carlyon
Coroner for Cornwall

The New Lodge Your ref:

Newquay Road , My ref:

Penmount po Date: 13 July 2016
Truro

TR4 9AA

Dear Dr Carlyon

 

A3075 Lambourne Mill Bend, Perranzabuloe

Thank you for your letter dated 24 May and attached report with respect to a road
traffic collision on the above highway.

The Council has now thoroughly investigated the circumstances of this unfortunate
incident and as we did not attend the inquest have made efforts to speak to the
Senior Investigation Officer and Collision Investigator. We have, of course, also
examined and reviewed all the accident data kept by the Council.

After careful consideration of the facts resulting from our investigations, advice from
the Council’s lead Road Safety Auditor, the content of your report and the geometry of
this particular highway, the Council has concluded that any works at this site such as
crash barriers or Cornish hedging are as likely to cause injury to 

In [8]:
from pdf2image import convert_from_path
import pytesseract

pdf_path = "downloads/2016-0230-Response-from-Cornwall-Council.pdf"

# Convert first page to image
images = convert_from_path(pdf_path, dpi=300)
images[0].show()  # display first page


In [9]:
# Run OCR manually on the first page
ocr_text = pytesseract.image_to_string(images[0])
print(ocr_text.strip() or "❌ No OCR output")


one and all onen hag olf

CORNWALL
COUNCIL

   

Dr E Carlyon
Coroner for Cornwall

 

The New Lodge Your ref:

Newquay Road a My ref:

Penmount yp Date: 13 July 2016
Truro

TR4 9AA

 

Dear Dr Carlyon
A3075 Lambourne Mill Bend, Perranzabuloe

Thank you for your letter dated 24 May and attached report with respect to a road
traffic collision on the above highway.

The Council has now thoroughly investigated the circumstances of this unfortunate
incident and as we did not attend the inquest have made efforts to speak to the
Senior Investigation Officer and Collision Investigator. We have, of course, also
examined and reviewed all the accident data kept by the Council.

After careful consideration of the facts resulting from our investigations, advice from
the Council’s lead Road Safety Auditor, the content of your report and the geometry of
this particular highway, the Council has concluded that any works at this site such as
crash barriers or Cornish hedging are as likely to cause inju